### Basic SQL queries

Therefor we went ahead and trimmed it down quite a lot to make it a bit more manageable. You can find the reduced version of this dataset under the name 'database.sqlite' in the resources tab of this unit.

The database contains four tables: station, status, trip and weather

#### Connecting to the database
We start by importing the libraries that we will need
The first thing that we will do is connect to the database. We do this using the connect function.

In [1]:
import pandas as pd
import sqlite3
# Connect to the database:
db = sqlite3.connect("c2_database.sqlite")

The variable db is now a Connection object. We can now use the pandas function read_sql_query to read the results of a SQL query on db directly into a pandas DataFrame. You will see that this is extremely convenient and useful.

The general syntax will be as follows

In [2]:
def run_query(query):
    return pd.read_sql_query(query, db)

 Let’s try this out. Suppose we wanted to retrieve the names of the tables in our database. An important thing to know is that the table names are always listed in a column called tbl_name of a master table called sqlite_master. We can retrieve this column then as follows

We can do this as follows:

In [3]:
run_query("SELECT tbl_name FROM sqlite_master;")

,tbl_name
0,station
1,status
2,trip
3,weather


Let’s focus on the trip table. We can obtain the first five rows of this table using

In [4]:
run_query("SELECT * FROM trip LIMIT 5;")

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4100,1392,8/29/2013 9:57,South Van Ness at Market,66,8/29/2013 10:20,San Francisco Caltrain 2 (330 Townsend),69,319,Subscriber,94118
1,4133,1965,8/29/2013 10:17,San Francisco Caltrain 2 (330 Townsend),69,8/29/2013 10:50,Powell Street BART,39,316,Customer,
2,4213,931,8/29/2013 11:14,Grant Avenue at Columbus Avenue,73,8/29/2013 11:30,Post at Kearney,47,473,Customer,20002
3,4282,572,8/29/2013 11:54,San Francisco Caltrain (Townsend at 4th),70,8/29/2013 12:04,Townsend at 7th,65,538,Customer,37206
4,4308,1000,8/29/2013 12:04,Redwood City Caltrain Station,22,8/29/2013 12:21,Redwood City Caltrain Station,22,159,Subscriber,94061


If we wanted only specific columns we can specify these in a comma-separated list as

In [5]:
run_query("SELECT start_station_name, end_station_name, duration FROM trip LIMIT 5;")

,start_station_name,end_station_name,duration
0,South Van Ness at Market,San Francisco Caltrain 2 (330 Townsend),1392
1,San Francisco Caltrain 2 (330 Townsend),Powell Street BART,1965
2,Grant Avenue at Columbus Avenue,Post at Kearney,931
3,San Francisco Caltrain (Townsend at 4th),Townsend at 7th,572
4,Redwood City Caltrain Station,Redwood City Caltrain Station,1000


We can filter this data further by using the WHERE keyword. Suppose we were interested only in the rows that have a specific start station. We can select these as follows



In [6]:
query = '''
SELECT start_station_name, end_station_name, duration FROM trip 
WHERE start_station_name = 'Paseo de San Antonio' 
LIMIT 5;
'''

We just wrote the command over several lines here for better clarity.

Executing this we get:

In [7]:
run_query(query)

,start_station_name,end_station_name,duration
0,Paseo de San Antonio,Paseo de San Antonio,539
1,Paseo de San Antonio,St James Park,5334
2,Paseo de San Antonio,Paseo de San Antonio,18009
3,Paseo de San Antonio,San Salvador at 1st,226
4,Paseo de San Antonio,San Salvador at 1st,256


We can also use boolean conditions with the WHERE keyword. Here is an example



In [9]:
query='''
SELECT start_station_name, end_station_name, duration FROM trip 
WHERE duration<100 
LIMIT 5
'''
run_query(query)

,start_station_name,end_station_name,duration
0,Market at Sansome,Market at Sansome,75
1,Market at 10th,South Van Ness at Market,94
2,Powell at Post (Union Square),Powell at Post (Union Square),82
3,Harry Bridges Plaza (Ferry Building),Harry Bridges Plaza (Ferry Building),60
4,Townsend at 7th,Townsend at 7th,83


And we can join multiple conditions using operators AND, OR or NOT. So if we were interested in the top five entries that start at the station Market at 10th and have duration less than 100 we can select them as follows



In [10]:
query='''
SELECT start_station_name, end_station_name, duration FROM trip 
WHERE (duration<100) AND (start_station_name = 'Market at 10th')
LIMIT 5;
'''
run_query(query)

,start_station_name,end_station_name,duration
0,Market at 10th,South Van Ness at Market,94
1,Market at 10th,Market at 10th,91
2,Market at 10th,Market at 10th,80
3,Market at 10th,Market at 10th,74
4,Market at 10th,Market at 10th,69


There are also a few additional operators that we can use with the WHERE keyword. For example BETWEEN lets us specify a range of values for a numerical column, and IN lets us specify a set of values.

One last keyword that we will look at is ORDER BY. Here we can include one or more column names that specify the sort order. By default, the order is ascending and if we want descending we must include the DESC keyword after the column name.

Let’s get the top five longest trips that started from the station Paseo de San Antonio.

In [11]:
query='''
SELECT start_station_name, end_station_name, duration FROM trip 
WHERE start_station_name = 'Paseo de San Antonio' 
ORDER BY duration DESC 
LIMIT 5;
'''
run_query(query)

,start_station_name,end_station_name,duration
0,Paseo de San Antonio,MLK Library,167156
1,Paseo de San Antonio,Paseo de San Antonio,70017
2,Paseo de San Antonio,San Salvador at 1st,53153
3,Paseo de San Antonio,Paseo de San Antonio,18009
4,Paseo de San Antonio,Paseo de San Antonio,11519


The SELECT statement has several additional extensions and options which can be added for greater functionality specific to your needs. Here we just covered the most common ones to help you understand the overall structure. In the next unit you will get to practice your skills with an exercise.

### Advanced SQL queries

In this unit we will continue working with the database db from the previous units and look at some more advanced SQL queries.

First things first, let’s again connect to the SQL database and create a helper function for queries:

In [12]:
import pandas as pd
import sqlite3

db = sqlite3.connect("c2_database.sqlite")

def run_query(query):
    return pd.read_sql_query(query, db)

#### Aggregate Functions

When using a SELECT statement we can include other functions inside the statement that act on the returned data. Some examples include SUM, AVG, MIN, MAX. Including these inside the query is more efficient than performing them on the returned database as it can often save both space and time. Let’s look at an example

Suppose we wanted to obtain the average duration of a trip. We have to use the option AVG and we must specify which column we want to apply this function to. In our case, it is the duration column. We can also specify the header of the column in the returned DataFrame using the AS keyword. The full query is then as follows

In [13]:
run_query("SELECT AVG(duration) AS 'Average Duration' FROM trip;")

,Average Duration
0,1070.969428


#### GROUP BY

We can separate the rows of the returned data using the GROUP BY command. Suppose we wanted to obtain the average duration of a trip for each type of subscription type. We can do this as follows

In [14]:
query='''
SELECT subscription_type, AVG(duration) AS 'Average Duration'  FROM trip 
GROUP BY subscription_type;
'''
run_query(query)

,subscription_type,Average Duration
0,Customer,3715.673978
1,Subscriber,580.480241


We can see that we have two subscription types customer and subscriber, and on average customers take much longer trips.

#### JOIN

All of the queries we looked at so far focused on a single table of our database trip. But often when working with databases we will be interested in questions which can be only answered by looking at multiple tables. For example, suppose we wanted to compare the number of trips per city. How would we do this? In the table trips we do not have any information about the cities. However, in the table station we do have a column called city that gives the city of each station. But how do we match up the two tables? Well, we would like to match the column start_station_name from the table trip with the column name from the table station. This is exactly what the JOIN statement lets us do.

The syntax will look like this

INNER JOIN station
ON trip.start_station_name = station.name

You might recall the idea an inner join from earlier. It just means that we are only interested in selecting the rows which match in both columns. Now we could have equivalently written

INNER JOIN trip
ON station.name = trip.start_station_name

We must include either version of this statement in our overall query. So let’s recall that we want to count the number of trips per city. So we must use some of the same techniques as before. First, we must group the data by city and then we must count the number of rows in each group. Let’s give this a go:


In [16]:
query = '''
SELECT station.city AS "City", COUNT(*) AS "Count"
FROM trip 
INNER JOIN station
ON trip.start_station_name = station.name
GROUP BY station.city;
'''
run_query(query)

,City,Count
0,Mountain View,2232
1,Palo Alto,853
2,Redwood City,409
3,San Francisco,74304
4,San Jose,4748


Great! And for one last challenge: suppose now that we wanted to find the duration of the longest intercity trip (the longest strip that starts and ends in different cities). Again we will have to work with the two tables trip and station but this time we are interested in obtaining the city of both the start station and the end station. This means that we need to join the trip table to the station table twice: the first for matching up trip.start_station_name with station.name and the second for matching up trip.end_station_name with station_name. To distinguish between the two views of the station table we will give each an alias name.

What we mean is this: we first write

INNER JOIN station AS start
ON trip.start_station_name = start.name
To make the first match. And then we write

INNER JOIN station AS end
ON trip.end_station_name = end.name
So now we will have the column start.city giving the city where the trip starts and end.city giving the city where the trip ends. We can then select these two columns for our returned DataFrame together with the duration column by including

SELECT start.city AS "Start city", end.city AS "End city", trip.duration
FROM trip
We then want to filter the rows with the condition that the start and end cities must be different. So we will include

WHERE start.city != end.city
And finally, we will order the rows by duration in descending order and return the top row by including

ORDER BY duration DESC
LIMIT 1
Putting all these parts together we obtain our full command

In [17]:
query = '''
SELECT start.city AS "Start city", end.city AS "End city", trip.duration
FROM trip
INNER JOIN station AS start
ON trip.start_station_name = start.name
INNER JOIN station AS end
ON trip.end_station_name = end.name
WHERE start.city != end.city
ORDER BY duration DESC
LIMIT 1;
'''
run_query(query)

,Start city,End city,duration
0,Mountain View,San Francisco,179330


#### Subqueries

Subqueries can be used to build queries on top of each other. Here is a basic example. Suppose we start with the following query which just selects the record that have duration less than 160.

SELECT * FROM trip
WHERE duration < 160;
Now we can nest this query and give it a name as follows

(SELECT * FROM trip
WHERE duration < 160) sub
We can now add another query on top of it, filtering for example for all records that start at the station ‘Post at Kearney’

SELECT sub.*
FROM (SELECT * FROM trip
WHERE duration < 160) sub
WHERE start_station_name='Post at Kearney';
Executing this query will now return all records that start at the station ‘Post at Kearney’ and have duration less than 160


In [18]:
query = '''
SELECT sub.*
FROM (SELECT * FROM trip
WHERE duration < 160) sub
WHERE start_station_name='Post at Kearney';
'''
run_query(query)

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,17227,140,9/10/2013 20:37,Post at Kearney,47,9/10/2013 20:39,Washington at Kearney,46,395,Subscriber,94133
1,29181,115,9/20/2013 17:17,Post at Kearney,47,9/20/2013 17:19,Post at Kearney,47,345,Customer,
2,31420,156,9/23/2013 10:59,Post at Kearney,47,9/23/2013 11:01,Beale at Market,56,292,Subscriber,94115
3,97477,138,11/15/2013 12:54,Post at Kearney,47,11/15/2013 12:56,Market at Sansome,77,273,Subscriber,94107
4,115555,154,12/4/2013 19:00,Post at Kearney,47,12/4/2013 19:02,Washington at Kearney,46,314,Subscriber,94133
5,153585,141,1/14/2014 23:09,Post at Kearney,47,1/14/2014 23:11,Howard at 2nd,63,381,Subscriber,94105
6,188403,159,2/18/2014 18:43,Post at Kearney,47,2/18/2014 18:45,Washington at Kearney,46,631,Subscriber,94133


Now we could have of course achieved this without subqueries by simply using the query…

SELECT * FROM trip
WHERE start_station_name='Post at Kearney' AND duration<160;
…that we saw before. You will see some applications later on in an exercise where subqueries will indeed come in very handy. The important thing to remember when writing or reading subqueries is that the inner query inside the parentheses is executed first, and afterwards the outer query is run on the results returned by the inner one.